In [5]:
import pandas as pd
import numpy as np

# 加载数据
data = pd.read_excel("Q2_未来30天各分拣中心每小时预测货量数据_方法2.xlsx")



In [6]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpInteger


# 筛选出SC60分拣中心的数据
sc60_data = data[data['分拣中心'] == 'SC60']

# 定义班次时间范围
shifts = {
    "00:00-08:00": range(0, 8),
    "05:00-13:00": range(5, 13),
    "08:00-16:00": range(8, 16),
    "12:00-20:00": range(12, 20),
    "14:00-22:00": range(14, 22),
    "16:00-24:00": range(16, 24)
}

# 按日期和班次整理每小时数据
shift_data = {}
for shift, hours in shifts.items():
    mask = sc60_data['小时'].isin(hours)
    grouped = sc60_data.loc[mask].groupby('日期')['预测每小时货量'].sum()
    shift_data[shift] = grouped

# 初始化模型
model = LpProblem("Staff_Scheduling_SC60", LpMinimize)

# 定义变量
workers = range(1, 201)
dates = sc60_data['日期'].unique()
x = LpVariable.dicts("Regular", [(date, shift, worker) for date in dates for shift in shifts for worker in workers], 0, 1, LpBinary)
y = LpVariable.dicts("Temp", [(date, shift) for date in dates for shift in shifts], 0, None, LpInteger)

# 目标函数
model += lpSum(y.values())

# 添加约束
for shift, demands in shift_data.items():
    for date, demand in demands.items():
        model += (lpSum(25 * x[(date, shift, worker)] for worker in workers) + 20 * y[(date, shift)] >= demand, f"Cargo_Requirement_{date}_{shift}")

# 正式工出勤天数约束
for worker in workers:
    model += lpSum(x[(date, shift, worker)] for date in dates for shift in shifts) <= 0.85 * len(dates), f"Max_Work_{worker}"

# 正式工每天只能安排一个班次
for worker in workers:
    for date in dates:
        model += lpSum(x[(date, shift, worker)] for shift in shifts) <= 1, f"One_Shift_Per_Day_{worker}_{date}"



In [ ]:
# 求解模型
model.solve()




In [ ]:
# 创建和填充结果DataFrame
results_df = pd.DataFrame([
    {'分拣中心': 'SC60', '日期': date, '班次': shift, '出勤员工': f'正式工{worker}'}
    for date in dates for shift in shifts for worker in workers if x[(date, shift, worker)].varValue == 1
] + [
    {'分拣中心': 'SC60', '日期': date, '班次': shift, '出勤员工': f'临时工{int(y[(date, shift)].varValue)}人'}
    for date in dates for shift in shifts if y[(date, shift)].varValue > 0
])

# 显示部分结果
results_df.head(20)

In [ ]:
import pandas as pd

# 创建空的DataFrame用于存储输出结果
columns = ['分拣中心', '日期', '班次', '出勤员工']
results_df = pd.DataFrame(columns=columns)

# 遍历结果，填充DataFrame
for date in range(1, 31):
    for shift in shifts:
        # 添加正式工的出勤记录
        for worker in range(1, 201):
            if x[(date, shift, worker)].varValue == 1:
                results_df = results_df.append({
                    '分拣中心': 'SC60',
                    '日期': f"2023-12-{date:02d}",
                    '班次': shift,
                    '出勤员工': f'正式工{worker}'
                }, ignore_index=True)
        # 添加临时工的需要数量
        temp_workers = y[(date, shift)].varValue
        if temp_workers > 0:
            results_df = results_df.append({
                '分拣中心': 'SC60',
                '日期': f"2023-12-{date:02d}",
                '班次': shift,
                '出勤员工': f'临时工{int(temp_workers)}人'
            }, ignore_index=True)

# 显示结果
results_df.head(20)




In [ ]:
# 可以选择输出到CSV文件中
results_df.to_csv("Q4_表6输出.csv", index=False)